# 02) Cleaning

As we see later on, this clean dataset ends up evolving as I continue to feature engineer for different objectives.

In [56]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import datetime


pd.set_option('display.max_columns', None)

## Taxi Data Import/Clean

In [15]:
# Import and join data sets

folder = '../data/'
written = False

#def data_import(folder_path):  --->  uncomment this line to use
    global written
    files = os.listdir(folder_path)
    output_file = '../data/taxi.csv'
    for file in files:
        if len(file) <= 9:
            input_file = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(input_file, encoding='utf-8')
            except UnicodeDecodeError:
                try:
                    df = pd.read_csv(input_file, encoding='latin1')
                except UnicodeDecodeError:
                    try:
                        df = pd.read_csv(input_file, encoding='ISO-8859-1')
                    except Exception as e:
                        print(f"Error reading file {file}: {e}")
                        continue
            if not written:
                df.to_csv(output_file, index=False)
                written = True
            else:
                df.to_csv(output_file, index=False, header=False, mode='a')

data_import(folder)

In [20]:
#Reading in dataframe of 2022 trips
trips = pd.read_csv('../data/taxi.csv')

#Dropping Unnamed: 0 column and checking dtypes
trips = trips.drop(columns=['Unnamed: 0'])

In [17]:
#Checking for df shape and NaNs
print(trips.shape)
trips.isna().sum()

(1942494, 11)


request_datetime             0
on_scene_datetime       534798
pickup_datetime              0
dropoff_datetime             0
PULocationID                 0
DOLocationID                 0
trip_miles                   0
base_passenger_fare          0
tips                         0
driver_pay                   0
congestion_surcharge         0
dtype: int64

In [22]:
#Turning request_datetime into a datetime object
trips['request_datetime'] = pd.to_datetime(trips['request_datetime'], format='mixed')

#Creating a column of dates to turn into the index
trips['req_index'] = pd.to_datetime(trips['request_datetime']).dt.date

#Creating a column of just the time
trips['req_time'] = pd.to_datetime(trips['request_datetime']).dt.time

#Creating req_date column again so can merge later
trips['req_date'] = pd.to_datetime(trips['request_datetime']).dt.date
trips['req_date'] = pd.to_datetime(trips['req_date'], format='mixed')

#Setting index as date for possible forecasting
trips.set_index('req_index', inplace=True)
trips = trips.sort_index()

In [26]:
#Dropping on_scene_datetime since we only need request and dropoff

trips = trips.drop(columns=['on_scene_datetime'])
trips.isna().sum()                   

request_datetime        0
pickup_datetime         0
dropoff_datetime        0
PULocationID            0
DOLocationID            0
trip_miles              0
base_passenger_fare     0
tips                    0
driver_pay              0
congestion_surcharge    0
req_time                0
req_date                0
dtype: int64

In [27]:
#Converting pickup time to datetime and creating a just pickup time column
trips['pickup_datetime'] = pd.to_datetime(trips['pickup_datetime'], format='mixed')
trips['pickup_time'] = pd.to_datetime(trips['pickup_datetime']).dt.time

#Converting dropoff time to datetime
trips['dropoff_datetime'] = pd.to_datetime(trips['dropoff_datetime'], format='mixed')

#Creating a dropoff_date column
trips['dropoff_date'] = pd.to_datetime(trips['dropoff_datetime']).dt.date
trips['dropoff_time'] = pd.to_datetime(trips['dropoff_datetime']).dt.time

## Weather Data Import/Clean

In [29]:
#Importing Weather dataframe
weather = pd.read_csv('../data/nyc_weather_2022.csv')

#Dropping columsn I don't need
wth = weather.drop(columns=['name', 'tempmax', 'tempmin', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'icon', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'stations'])

#Converting datetime column to a pandas datetime object
wth['datetime'] = pd.to_datetime(wth['datetime'], format='mixed')

In [30]:
#Converting precipitation column to numerics

#Checking the values
wth['preciptype'].value_counts()

#Checking the NaNs
wth['preciptype'].isna().sum()

#Fillin NaNs with "missing" string in order to ordinal map
wth['preciptype'] = wth['preciptype'].fillna('missing')

#Ordinal Mapping preciptypes/combining
precip = {'missing': 0, 'rain': 1, 'snow': 2, 'rain,snow': 3, 
          'rain,freezingrain,snow': 4, 'rain,freezingrain,snow,ice': 5,
          'rain,freezingrain': 6, 'freezingrain': 6, 'freezingrain,snow,ice': 5, 'rain,snow,ice': 5}

wth['preciptype'] = wth['preciptype'].map(precip)

#Renaming weather datetime column for easy merging with trips df later
wth.rename(columns={'datetime': 'req_date'}, inplace=True)

#Converting that column to datetime object for easy merging with trips df later
wth['req_date'] = pd.to_datetime(wth['req_date'], format='mixed')

## Taxi Zone Data Import/Clean

In [31]:
#Importing Taxi Zones
zones = pd.read_csv('../data/taxi_zones.csv')

#Dropping columns I don't need
zones = zones.drop(columns=['OBJECTID', 'Shape_Leng', 'Shape_Area'])

#Casting LocationID as float for easy merging with trips df
zones['LocationID'] = zones['LocationID'].astype(float)

#Changing LocationID column name for easy merging with trips df
zones = zones.rename(columns={'LocationID':'PULocationID'})

In [32]:
#Making sure the value counts for trips and zones df line up
trips.PULocationID.value_counts(), zones.PULocationID.value_counts()

(PULocationID
 138    36200
 132    33762
 79     27803
 61     25928
 230    25345
        ...  
 111       65
 99        35
 2          6
 110        3
 199        2
 Name: count, Length: 260, dtype: int64,
 PULocationID
 103.0    3
 56.0     2
 1.0      1
 175.0    1
 181.0    1
         ..
 93.0     1
 94.0     1
 95.0     1
 96.0     1
 262.0    1
 Name: count, Length: 260, dtype: int64)

## Merging DFs

In [33]:
#Checking the shape of trips and weather df
trips.shape, wth.shape

((1942494, 15), (365, 3))

In [34]:
#Left merging trips and weather to ensure no trips data is lost
df = pd.merge(
    left=trips.reset_index(),
    right=wth,
    how='left',
    on='req_date'
)

#Checking shape of trips and the new df to ensure no trips data is lost
trips.shape, df.shape

((1942494, 15), (1942494, 18))

In [35]:
#Left merging df and zones as to not loose any df data and to assign each trip a zone
df = pd.merge(
    left=df,
    right=zones,
    how='left',
    on='PULocationID'
)

#Checking shape to see if we lost data
df.shape

(1947935, 21)

In [36]:
#Checking for missings due to undropped NaNs in wth and zones dataset
df.isna().sum()

req_index                 0
request_datetime          0
pickup_datetime           0
dropoff_datetime          0
PULocationID              0
DOLocationID              0
trip_miles                0
base_passenger_fare       0
tips                      0
driver_pay                0
congestion_surcharge      0
req_time                  0
req_date                  0
pickup_time               0
dropoff_date              0
dropoff_time              0
temp                      0
preciptype                0
the_geom                435
zone                    435
borough                 435
dtype: int64

In [37]:
#Seeing the 888s, I decided to see what those all are

#Isolating the NaNs of df by zone
na_zone = df[df['zone'].isna()]

#Looking to see what the values are that are missing
na_zone['PULocationID'].value_counts()

PULocationID
57     344
265     91
Name: count, dtype: int64

In [38]:
#Looking into these more and checking taxi zones online, I have discovered that there are actually no taxi zones 57.0 and 265.0, which resulted in 888 NaNs
zones.iloc[262]

the_geom        MULTIPOLYGON (((-73.94383256699986 40.78285908...
zone                                               Yorkville East
PULocationID                                                262.0
borough                                                 Manhattan
Name: 262, dtype: object

In [39]:
#Getting the index back after it was lost
df = df.reset_index()

#Resetting index to be dates for possible forecasting
df['req_index'] = pd.to_datetime(df['req_index'])
df = df.set_index('req_index')
df = df.drop(columns=['index'])

#Sorting df by date
df = df.sort_index()

#Dropping unnecessary columns
df = df.drop(columns=['request_datetime', 'pickup_datetime', 'dropoff_datetime', 'the_geom'])

#Filling borough NaNs with "missing" string for mapping
df['borough'] = df['borough'].fillna('missing')

#Renaming to keep names and create ordinal map
df = df.rename(columns={'borough': 'borough_name'})

#Ordinal mapping the boroughs
ordinals = {'missing': 0, 'Bronx': 1, 'Manhattan': 2, 'Queens': 3, 
          'Brooklyn': 4, 'Staten Island': 5}

#Creating new column
df['borough'] = df['borough_name'].map(ordinals)

In [40]:
#Ensuring my datetimeobjects are datetime objects and creating new columns

#Setting dropoff date
df['dropoff_date'] = pd.to_datetime(df['dropoff_date'], format='mixed')

#Creating request hour column
#df['req_hour'] = df['req_time'].dt.hour

#Formatting pickup and dropoff to get trip duration
df['pickup_time'] = pd.to_datetime(df['pickup_time'], format='%H:%M:%S').dt.floor('min')
df['dropoff_time'] = pd.to_datetime(df['dropoff_time'], format='%H:%M:%S').dt.floor('min')

# Calculate trip duration
df['trip_duration'] = df['dropoff_time'] - df['pickup_time']

#Converting trip duration timedelta type to seconds
df['trip_duration'] = df['trip_duration'].dt.seconds

#Creating a numeric month column
df['month'] = pd.to_datetime(df['req_date']).dt.month

#Creating a numeric day of month column
df['day'] = pd.to_datetime(df['req_date']).dt.day


In [54]:
#Creating a driver pay + tips column
df['driver_made'] = df['tips'] + df['driver_pay']

#Creating a request day of week
df['day_of_week'] = df['req_date'].dt.strftime('%A')

#Renaming day column to day_of_month
df = df.rename(columns={'day': 'day_of_month'})

#Converting trip_duration to minutes
df['trip_duration'] = df['trip_duration']/60

# Makine an hour column
df['hour'] = df['req_time'].apply(lambda x: x.hour)

# Making a minute column
df['minute'] = df['req_time'].apply(lambda x: x.minute)

,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time,temp,preciptype,zone,borough_name,borough,trip_duration,month,day_of_month,driver_made,day_of_week
req_index,,,,,,,,,,,,,,,,,,,,,,
2022-01-04,248,41,7.050,35.33,7.91,25.39,0.00,07:32:25,2022-01-04,1900-01-01 07:40:00,2022-01-04,1900-01-01 08:05:00,27.6,0,West Farms/Bronx River,Bronx,1,25.0,1,4,33.30,Tuesday
2022-01-04,216,130,2.470,12.93,0.00,8.33,0.00,07:24:46,2022-01-04,1900-01-01 07:30:00,2022-01-04,1900-01-01 07:41:00,27.6,0,South Ozone Park,Queens,3,11.0,1,4,8.33,Tuesday
2022-01-04,223,141,5.310,20.72,0.00,18.08,2.75,07:24:47,2022-01-04,1900-01-01 07:29:00,2022-01-04,1900-01-01 07:52:00,27.6,0,Steinway,Queens,3,23.0,1,4,18.08,Tuesday
2022-01-04,222,188,3.670,24.75,0.00,17.34,0.00,07:24:47,2022-01-04,1900-01-01 07:34:00,2022-01-04,1900-01-01 07:53:00,27.6,0,Starrett City,Brooklyn,4,19.0,1,4,17.34,Tuesday
2022-01-04,263,233,2.853,10.94,0.00,7.57,2.75,07:24:47,2022-01-04,1900-01-01 07:28:00,2022-01-04,1900-01-01 07:36:00,27.6,0,Yorkville West,Manhattan,2,8.0,1,4,7.57,Tuesday


In [67]:
#Dropping columns no longer needed

df = df.drop(columns=['PULocationID', 'DOLocationID', 'base_passenger_fare',
        'driver_pay', 'req_time', 'req_date', 'pickup_time', 'dropoff_date', 'dropoff_time', 'borough'])

df.head()

,trip_miles,tips,congestion_surcharge,temp,preciptype,zone,borough_name,trip_duration,month,day_of_month,driver_made,day_of_week,hour,minute
req_index,,,,,,,,,,,,,,
2022-01-04,7.050,7.91,0.00,27.6,0,West Farms/Bronx River,Bronx,25.0,1,4,33.30,Tuesday,7,32
2022-01-04,2.470,0.00,0.00,27.6,0,South Ozone Park,Queens,11.0,1,4,8.33,Tuesday,7,24
2022-01-04,5.310,0.00,2.75,27.6,0,Steinway,Queens,23.0,1,4,18.08,Tuesday,7,24
2022-01-04,3.670,0.00,0.00,27.6,0,Starrett City,Brooklyn,19.0,1,4,17.34,Tuesday,7,24
2022-01-04,2.853,0.00,2.75,27.6,0,Yorkville West,Manhattan,8.0,1,4,7.57,Tuesday,7,24


In [ ]:
#Discovered in EDA that there are negative values in terms of "driver_made", which are probably typos since drivers aren't losing money on trips
df[df['driver_made'] < 0]

#Changing that now and rexporting
df['driver_made'] = df['driver_made'].abs()

#Checking
df[df['driver_made' < 0]]

In [69]:
#Exporting df
df.to_csv('../data/clean/041324_taxi_recs.csv')